# Systems Identification Model Fitting

Fit a Systems Identification model off based off of this [specification](https://hackmd.io/w-vfdZIMTDKwdEupeS3qxQ) and [spec](https://hackmd.io/XVaejEw-QaCghV1Tkv3eVQ) with data obtained in [data_acquisition.ipynb](data/data_acquisition.ipynb).

## Analyze and Prepare Data


In [415]:
# import libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

states = pd.read_csv('data/states.csv')
del states['Unnamed: 0']
states.head()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RAIInUniswapV2(RAI/ETH),collateral,debt,ETH Price (OSM)
0,0.000908,3.028273,12374951.0,0.0,2.634893e+07,2.665497e+07,1.157921e+32,228549.463344,590.0,0.962694,0.999996,0.999965,3.006316,1.393020e+07,1249.4226,293469.366076,3355.820670
1,0.000910,3.015545,12374655.0,0.0,2.631873e+07,2.662478e+07,1.157921e+32,228491.482327,590.0,0.962694,0.999996,0.999965,3.006330,1.393919e+07,1249.4226,293469.366076,3376.582733
2,0.000910,3.015545,12374385.0,0.0,2.631954e+07,2.662558e+07,1.157921e+32,228491.482327,592.0,0.948182,0.999994,0.999951,3.006348,1.394005e+07,1249.4226,293469.366076,3373.697700
3,0.000910,3.015545,12374112.0,0.0,2.631954e+07,2.662606e+07,1.157921e+32,228397.412129,592.0,0.948182,0.999994,0.999951,3.006366,1.394005e+07,1249.4226,293469.366076,3380.394789
4,0.000905,3.052556,12373874.0,0.0,2.645954e+07,2.676545e+07,1.157921e+32,228397.412129,592.0,0.948182,0.999994,0.999951,3.006384,1.395508e+07,1249.4226,293469.366076,3389.969206


In [416]:
# save block numbers
blockNumbers = states.block_number.values

In [417]:
states['RedemptionPriceinEth'] = states['RedemptionPrice'] / states['ETH Price (OSM)']


### Mapping of specification states to data
The quantity state variables of the system are:

* ETH in collateral = collateral
* ETH in Uniswap = debt?
* RAI in Uniswap = RAIInUniswapV2(RAI/ETH)
* RAI drawn from SAFEs =?

The metric state variables of the system are:

* Market Price of RAI in ETH = marketPriceEth
* Market Price of RAI in USD = marketPriceUsd
* Market Price of ETH in USD = ETH Price (OSM)

The metric control variables of the system are:

* Redemption Price of RAI in USD = RedemptionPrice
* Redemption Price of RAI in ETH = RedemptionPriceinEth Calculate based off of ETH Price (OSM)?

### Model Formulation
Create X array of:
* Current state: $x$
* $\Delta$ in state: $u_t=x^+-x$
* posterior state $x^+$

Y is the next timestep $\Delta$ in state.

## Questions:
* Do I create a linear regression model for each signal individually? I.e. marketPriceEth
* What would RAI drawn from SAFEs be?


**I wired this example up as X is the previous state, previous state change, current state, and we are predicting in the new state change for the signal marketPriceEth - is this correct?** 

## Create model

In [418]:
def univariate_identification(states_df,state):
    '''
    Function to create systems identification model
    
    Parameters:
    states_df: Pandas dataframe with states information
    state: string of state name
    
    returns fitted_model
    '''
    univariate_state = states_df[[state]]
    univariate_state['delta'] = univariate_state.diff()
    univariate_state['posterior'] = univariate_state[state] + univariate_state['delta']
    univariate_state['Y_state_change'] = univariate_state.delta.shift(-1)
    univariate_state.dropna(inplace=True)
    print('DataFrame Head')
    print(univariate_state.head(10))
    print('\n DataFrame Tail')
    print(univariate_state.tail(10))
    
    # create model
    split_point = int(len(univariate_state) * .8)
    train = univariate_state.iloc[0:split_point]
    test = univariate_state.iloc[split_point:]
    
    Y = train['Y_state_change']
    X = train[[state,'delta','posterior']]
    
    model = sm.OLS(Y,X)

    fitted_model = model.fit_regularized(method='sqrt_lasso')
    fitted_model = model.fit()
    print('\n Model Parameters')
    print(fitted_model.params)
    
    Y_test = test['Y_state_change']
    X_test = test[[state,'delta','posterior']]
    predicted = fitted_model.predict(X_test)
    print('\n Model RMSE:')
    print(sm.tools.eval_measures.rmse(Y_test, predicted, axis=0))
    
    return fitted_model

In [419]:
marketPriceEth_model = univariate_identification(states,'marketPriceEth')

DataFrame Head
    marketPriceEth     delta  posterior  Y_state_change
1         0.000910  0.000001   0.000911        0.000000
2         0.000910  0.000000   0.000910        0.000000
3         0.000910  0.000000   0.000910       -0.000005
4         0.000905 -0.000005   0.000900        0.000000
5         0.000905  0.000000   0.000905        0.000009
6         0.000913  0.000009   0.000922        0.000000
7         0.000913  0.000000   0.000913        0.000003
8         0.000916  0.000003   0.000919        0.000006
9         0.000922  0.000006   0.000928        0.000000
10        0.000922  0.000000   0.000922        0.000000

 DataFrame Tail
      marketPriceEth     delta  posterior  Y_state_change
1887        0.001772  0.000000   0.001772        0.000000
1888        0.001772  0.000000   0.001772        0.000000
1889        0.001772  0.000000   0.001772       -0.000065
1890        0.001707 -0.000065   0.001642        0.001435
1891        0.003142  0.001435   0.004577        0.000000
1892